In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **Imports**

In [18]:
from getModel import blModel, eeModel, eeModel_V0, eeModel_V1, eeModel_V2, eeModel_V3
from trainerv2 import Handler

# training tools
from model_builder.core.metrics import multiclass_accuracy
from torch.nn.functional import cross_entropy

# data tools
import numpy as np
import torch
import pickle

# **Data**

In [19]:
with open('./Data/RML22.pickle.01A', 'rb') as f:
    data = pickle.load(f, encoding="latin1")

In [20]:
mods = [lst[0] for lst in data.keys()]
snrs = [lst[1] for lst in data.keys()]

In [21]:
len(mods), len(snrs)

(210, 210)

In [23]:
dataset = []
for mod, snr in zip(mods, snrs):
    # print(data[(mod, snr)].shape, mod, snr)
    dataset.extend(data[(mod, snr)])

dataset = np.array(dataset)
dataset.shape  # (210*2000, 2, 128)

(420000, 2, 128)

In [7]:
num_classes = len(set(mods))
name2label = {name: i for i, name in enumerate(np.unique(mods))}
labels = torch.from_numpy(np.array([name2label[name] for name in mods]))
labels = labels.repeat_interleave(2000)

In [8]:
class IQDataset:
    def __init__(self, x, y, transform_x=None, transform_y=None):
        self.x = x
        self.y = y
        self.transform_x = transform_x
        self.transform_y = transform_y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        if self.transform_x:
            x = self.transform_x(x)
        elif isinstance(x, np.ndarray):
            x = torch.from_numpy(x)
        if self.transform_y:
            y = self.transform_y(y)
        elif isinstance(y, np.ndarray):
            y = torch.from_numpy(y)
        return x, y

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=2, test_size=0.4, random_state=0)
dev_idx, test_idx = next(sss.split(dataset, labels))
train_idx, val_idx = next(sss.split(dataset[dev_idx], labels[dev_idx]))

train_data = IQDataset(
    dataset[train_idx], labels[train_idx])
val_data = IQDataset(
    dataset[val_idx], labels[val_idx])
test_data = IQDataset(
    dataset[test_idx], labels[test_idx])

len(train_data), len(val_data), len(test_data)

(151200, 100800, 168000)

In [10]:
one_x, one_y = train_data[0]
one_x.shape, one_y.shape

(torch.Size([2, 128]), torch.Size([]))

In [11]:
NUM_EPOCHS = 50

# **Baseline Model**

In [12]:
blNet = blModel()
print(blNet)

blModel(
  (baseModel): Sequential(
    (0): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
  )
  (longBranch): Sequential(
    (0): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(32, 

In [13]:
hdl = Handler(blNet, nepochs=NUM_EPOCHS, crit=cross_entropy,
              sch="cosine", modelPath='./Models/', modelName="blModel")
hdl.train(train_data, val_data, bs=64, lr=1e-3,
          wd=1e-4, metric_name='val_loss')

Better model found at epoch 0 with val_loss value: 1.1580520868301392.
Better model found at epoch 2 with val_loss value: 1.128730058670044.
Better model found at epoch 4 with val_loss value: 1.124612808227539.
Better model found at epoch 6 with val_loss value: 1.121091604232788.
Better model found at epoch 8 with val_loss value: 1.1149768829345703.
Reducing LR to 0.0001.
Better model found at epoch 12 with val_loss value: 1.1142569780349731.
Better model found at epoch 14 with val_loss value: 1.114080786705017.
Better model found at epoch 17 with val_loss value: 1.1122633218765259.
Better model found at epoch 18 with val_loss value: 1.1080347299575806.
Better model found at epoch 19 with val_loss value: 1.1050848960876465.
Reducing LR to 1e-05.
Reducing LR to 1.0000000000000002e-06.
Reducing LR to 1.0000000000000002e-07.
Reducing LR to 1.0000000000000004e-08.
Reducing LR to 1.0000000000000005e-09.
Reducing LR to 1.0000000000000006e-10.
Reducing LR to 1.0000000000000006e-11.
Reducing L

Validation Loss: 1.10531 Multiclass Accuracy: 0.498
Model saved at: Models/blModel.pt


# **EE Model**

In [14]:
eev0Net = eeModel_V0()
print(eev0Net)

one_y1, one_y2 = eev0Net(one_x.unsqueeze(0))
one_y1.shape, one_y2.shape

Number of base parameters: 576
Number of short branch parameters: 68010
Number of long branch parameters: 118842
Difference = 50832
eeModel_V0(
  (baseModel): Sequential(
    (0): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (shortBranch): Sequential(
    (0): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=1024, out_features=64, bias=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=32, out_features=10, bias=True)
  )
  (longBranch): Sequential(
    (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, mome

(torch.Size([1, 10]), torch.Size([1, 10]))

In [15]:
def loss_func(outputs, target, reduction="mean"):
    out1, out2 = outputs
    return (cross_entropy(out1, target, reduction=reduction) + cross_entropy(out2, target, reduction=reduction)) / 2


def accuracy(outputs, target):
    out1, out2 = outputs
    return (multiclass_accuracy(out1, target) + multiclass_accuracy(out2, target)) / 2

In [16]:
hdl = Handler(eev0Net, nepochs=NUM_EPOCHS, crit=loss_func,
              sch="cosine", metrics=[accuracy], modelPath='./Models/', modelName="eev0Model")
hdl.train(train_data, val_data, bs=64, lr=1e-3,
          wd=1e-4, metric_name="val_loss")

Better model found at epoch 0 with val_loss value: 1.4748704433441162.
Better model found at epoch 2 with val_loss value: 1.4322011470794678.
Better model found at epoch 3 with val_loss value: 1.3850140571594238.
Better model found at epoch 4 with val_loss value: 1.3373169898986816.
Better model found at epoch 6 with val_loss value: 1.3231210708618164.
Reducing LR to 0.0001.
Reducing LR to 1e-05.
Reducing LR to 1.0000000000000002e-06.
Reducing LR to 1.0000000000000002e-07.
Reducing LR to 1.0000000000000004e-08.
Reducing LR to 1.0000000000000005e-09.
Reducing LR to 1.0000000000000006e-10.
Reducing LR to 1.0000000000000006e-11.
Reducing LR to 1.0000000000000006e-12.
Reducing LR to 1.0000000000000007e-13.
Reducing LR to 1.0000000000000008e-14.
Reducing LR to 1.0000000000000009e-15.
Reducing LR to 1.000000000000001e-16.
Reducing LR to 1.000000000000001e-17.


Training completed




Validation Loss: 1.32317 Accuracy: 0.389
Model saved at: Models/eev0Model.pt
